In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext

In [2]:
zillow = pd.read_csv('~\Calc\Data\Zillow.csv')

In [3]:
zillow = pd.DataFrame(data = zillow)

In [4]:
zillow

,PropertyID,Street,City,State,ZipCode,Beds,Baths,SQFT,Price,Down Payment,Monthly Payment,House Type,Year Built,Heating,Cooling,Parking,Lot,Price/SQFT
0,1,243 Thisbe Ave,Cincinnati,OH,45233,2.0,1.0,744,130000.0,26000.0,772.51,Single Family Residence,1931,Forced Air,Central Air,Open Parking,0.26,175.0


In [5]:
street = str(zillow['Street'][0]).replace(' ', '-').replace('0----', '')
city = str(zillow['City'][0]).replace(' ', '-').replace('0----', '')
state = str(zillow['State'][0]).replace(' ', '-').replace('0----', '')
zipcode = str(zillow['ZipCode'][0]).replace(' ', '-').replace('0----', '')
address = street.replace('-', ' ') + ", " + city + ", " + state + ", " + zipcode

In [6]:
https://www.zillow.com/homedetails/335-W-McMillan-St-Cincinnati-OH-45219/170588118_zpid/

SyntaxError: invalid syntax (<ipython-input-6-28362a8e0703>, line 1)

In [7]:
city = 'Cincinnati'
state = 'OH'

In [22]:
request_headers = {
    'accept': 
    'text/html, application/xhtml+xml, application/xml;q=0.9, image/webp, image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US, en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
with requests.Session() as session:
    url= "https://www.mashvisor.com/cities/" + state.lower().replace(' ', '') + "/" + city.replace(' ', '') + "-investment-property-guide"
    response = session.get(url, headers = request_headers)
print(url)

https://www.mashvisor.com/cities/oh/Cincinnati-investment-property-guide


In [23]:
soup = BeautifulSoup(response.content, 'html.parser')
print(str(soup))

<!DOCTYPE html>

<html>
<head>
<title>
        Investment Property in Cincinnati, OH | Real Estate Investing in Cincinnati
    </title>
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<meta content="Investment Property in Cincinnati, OH | Real Estate Investing in Cincinnati" name="title"/>
<meta content="Search for the top real estate investments in Cincinnati, Ohio. Find traditional &amp; Airbnb Cincinnati investment property based on cash flow, cash on cash return and cap rate calculations. Median Property Sale Price: $239,900, 7.01 Aribnb and 2.14 Traditional Cash on Cash Return." name="description"/>
<meta content="real estate investing, market analysis, real estate, market trends, Cincinnati real estate, investment properties, Real Estate Investment Properties Cincinnati, Airbnb Cincinnati, Best Real Estate Investments, Buy and Hold" name="keywords"/>
<meta content="Investment Property 

In [56]:
table = pd.read_html(url) # Returns list of all tables on page
table = float(table[0].iloc[0].iloc[2].replace('$ ', '').replace(',', ''))
table

2070.0

## Occupancy Rate ##

In [15]:
occupancyRate = [a for a in (td.find('h3', {'class': 'text-ment'}) for td in soup.findAll('section', {'class': 'occ-rate col-md-1 col-sm-12 col-xs-12 no-padding inner-section'})) if a]
occupancyRate = str(occupancyRate)
occupancyRate = occupancyRate.split('>')[1]
occupancyRate = occupancyRate.split('<')[0]
occupancyRate = '\nN/A\n'
if occupancyRate == '\nN/A\n':
    occupancyRate = None
else:
    occupancyRate = re.findall(r'\d+', occupancyRate)[0]
    occupancyRate = float(occupancyRate)
print(occupancyRate)

None


In [32]:
#airbnbincome = [a for a in (td.find_all('td') for td in soup.find_all('div', {'class': 'rent-prices-section col-md-12 col-sm-12 col-xs-12 no-padding'}))][0]
airbnbincome = soup.find_all('td')
airbnbincome[0]
airbnbincome = [a for a in (td.find_all('td') for td in soup.find_all('section', {'class': 'rent-price-content small-content table-responsive hidden-sm hidden-md hidden-lg hidden-xl'}))][0][0]
airbnbincome

<td>
$
1,124
</td>

## Airbnb Income ##

In [24]:
beds= 4
if (beds == '1'):
    airbnbincome = [a for a in (td.find_all('td') for td in soup.findAll('div', {'class': 'rent-price-content small-content table-responsive hidden-sm hidden-md hidden-lg hidden-xl'}))][0]
    airbnbincome = airbnbincome[0].text
    airbnbincome = float(re.findall(r'\d,\d+', airbnbincome)[0].replace(',', ''))
elif (beds == '2'):
    airbnbincome = [a for a in (td.find_all('td') for td in soup.findAll('div', {'class': 'rent-prices-section col-md-12 col-sm-12 col-xs-12 no-padding'}))][0]
    print(airbnbincome)
    airbnbincome = airbnbincome[1].text
    airbnbincome = float(re.findall(r'\d,\d+', airbnbincome)[0].replace(',', ''))
elif (beds == '3'):
    airbnbincome = [a for a in (td.find_all('td') for td in soup.findAll('div', {'class': 'rent-prices-section col-md-12 col-sm-12 col-xs-12 no-padding'}))][0]
    airbnbincome = airbnbincome[2].text
    airbnbincome = float(re.findall(r'\d,\d+', airbnbincome)[0].replace(',', ''))
elif beds == 4:
    airbnbincome = [a for a in (td.find_all('td') for td in soup.findAll('div', {'class': 'rent-prices-section col-md-12 col-sm-12 col-xs-12 no-padding'}))][0]
    airbnbincome = airbnbincome[3].text
    airbnbincome = float(re.findall(r'\d,\d+', airbnbincome)[0].replace(',', ''))
airbnbincome

4023.0

## Airbnb ##

In [107]:
airbnb = pd.DataFrame(columns = ['PropertyID', 'AirBnb', 'Occupancy Rate'])
airbnb = airbnb.append({'PropertyID': 1, 'AirBnb': airbnbincome, 'Occupancy Rate': occupancyRate}, ignore_index = True)

In [108]:
airbnb.to_csv('~\Calc\Data\Airbnb.csv', index = False)

In [109]:
airbnb

,PropertyID,AirBnb,Occupancy Rate
0,1.0,2949.0,55.0


In [110]:
df = pd.read_csv('~\Calc\Data\AllData.csv')

In [111]:
airbnb = pd.merge(airbnb, df, on = "PropertyID")

In [91]:
airbnb.to_csv('~\Calc\Data\AllData.csv', index = False)

In [94]:
airbnb

,PropertyID,AirBnb,Occupancy Rate,Street,City,State,ZipCode,Beds,Baths,SQFT,Price,Down Payment,Monthly Payment,House Type,Year Built,Heating,Cooling,Parking,Lot,Price/SQFT
0,1.0,2070.0,59.0,335 W McMillan St,Cincinnati,OH,45219,4,2,NaN,299000.0,59800.0,1417.98,MultiFamily,1908,Natural Gas'],Window Unit(s),No Garage,NaN,NaN
